In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wfdb

In [18]:
diretorio = '/home/matheus/Documentos/Facul/IC/ECG_classification/data/'

In [37]:
# Lista com os códigos que precisamos
codigo = [10370003] # PR e LPR

# Lista dos leads
leads = [0,2]

In [20]:
data_frame = pd.read_csv('Total.csv')
data_frame

,fs,sig_len,n_sig,base_date,base_time,units,sig_name,Age,Sex,Dx,Rx,Hx,Sx,arquivos,base
0,500,5000,1,NaN,00:00:24,['mV'],['II'],47.0,Female,426177001,Unknown,Unknown,Unknown,JS04442,WFDB_ChapmanShaoxing
1,500,5000,1,NaN,00:00:23,['mV'],['II'],41.0,Female,426177001,Unknown,Unknown,Unknown,JS00992,WFDB_ChapmanShaoxing
2,500,5000,1,NaN,00:00:23,['mV'],['II'],68.0,Female,427084000,Unknown,Unknown,Unknown,JS02338,WFDB_ChapmanShaoxing
3,500,5000,1,NaN,00:00:23,['mV'],['II'],41.0,Female,426177001,Unknown,Unknown,Unknown,JS01381,WFDB_ChapmanShaoxing
4,500,5000,1,NaN,00:00:23,['mV'],['II'],70.0,Male,426177001,Unknown,Unknown,Unknown,JS02487,WFDB_ChapmanShaoxing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158334,257,462600,1,NaN,00:00:15,['mv'],['II'],59.0,Female,251170000,Unknown,Unknown,Unknown,I0020,WFDB_StPetersburg
158335,257,462600,1,NaN,00:00:15,['mv'],['II'],73.0,Male,164873001,Unknown,Unknown,Unknown,I0072,WFDB_StPetersburg
158336,257,462600,1,NaN,00:00:15,['mv'],['II'],73.0,Male,53741008,Unknown,Unknown,Unknown,I0072,WFDB_StPetersburg
158337,257,462600,1,NaN,00:00:15,['mv'],['II'],73.0,Male,427084000,Unknown,Unknown,Unknown,I0072,WFDB_StPetersburg


In [21]:
# Cria tabela com apenas os códigos necessários
df = pd.DataFrame(columns= ['arquivos', 'base', 'Dx'])
for x in range(len(codigo)):
    df = df.append(data_frame.loc[lambda data_frame: data_frame['Dx'] == codigo[x]])


In [22]:
df = df.reset_index()
df


,index,arquivos,base,Dx,fs,sig_len,n_sig,base_date,base_time,units,sig_name,Age,Sex,Rx,Hx,Sx
0,29541,Q1033,WFDB_CPSC2018_2,10370003,500.0,15500.0,1.0,NaN,00:00:15,['mV'],['II'],59.0,Female,Unknown,Unknown,Unknown
1,31006,Q0398,WFDB_CPSC2018_2,10370003,500.0,9000.0,1.0,NaN,00:00:15,['mV'],['II'],82.0,Male,Unknown,Unknown,Unknown
2,31872,Q1807,WFDB_CPSC2018_2,10370003,500.0,12500.0,1.0,NaN,00:00:15,['mV'],['II'],65.0,Female,Unknown,Unknown,Unknown
3,32363,JS23448,WFDB_Ningbo,10370003,500.0,5000.0,1.0,NaN,00:00:27,['mV'],['II'],78.0,Female,Unknown,Unknown,Unknown
4,32417,JS23137,WFDB_Ningbo,10370003,500.0,5000.0,1.0,NaN,00:00:26,['mV'],['II'],86.0,Female,Unknown,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868,157445,HR07001,WFDB_PTBXL,164947007,500.0,5000.0,1.0,NaN,00:00:04,['mv'],['II'],79.0,Male,Unknown,Unknown,Unknown
1869,157620,HR01756,WFDB_PTBXL,164947007,500.0,5000.0,1.0,NaN,00:00:04,['mv'],['II'],73.0,Female,Unknown,Unknown,Unknown
1870,157648,HR12587,WFDB_PTBXL,164947007,500.0,5000.0,1.0,NaN,00:00:04,['mv'],['II'],79.0,Male,Unknown,Unknown,Unknown
1871,157914,HR16305,WFDB_PTBXL,164947007,500.0,5000.0,1.0,NaN,00:00:04,['mv'],['II'],75.0,Male,Unknown,Unknown,Unknown


In [23]:
#Cria a tabela dos records
records = pd.DataFrame()

In [38]:
for x in range(len(df)):
    destino = diretorio + df.loc[x]['base'] + '/' + df.loc[x]['arquivos']
    record, fields = wfdb.rdsamp(destino, channels = leads)
    # Colocar o record em uma data_frame Junto com seu código
    aux = pd.DataFrame(record.transpose())
    # Inserir a linha com o nome da arritmia
    aux['arritmia'] = int(df.loc[x]['Dx'])
    # Inserir coluna de leads
#    aux['leads'] = leads
    # Inserir a coluna com nome do arquivo
    aux['arquivo'] = df.loc[x]['arquivos']
    records = records.append(aux)

In [40]:
records

,0,1,2,3,4,5,6,7,8,9,...,15495,15496,15497,15498,15499,15500,15501,arquivo,arritmia,leads
0,0.000,1.037000e+07,0.063,0.061,0.060,0.058,0.058,0.057,0.056,0.054,...,0.012,0.011,0.009,0.008,0.011,0.014,0.018,NaN,NaN,NaN
1,1.000,1.037000e+07,-0.021,-0.022,-0.021,-0.022,-0.022,-0.021,-0.020,-0.021,...,-0.012,-0.011,-0.012,-0.011,-0.014,-0.014,-0.015,NaN,NaN,NaN
0,0.000,1.037000e+07,0.153,0.151,0.146,0.143,0.142,0.140,0.138,0.137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.000,1.037000e+07,-0.064,-0.053,-0.042,-0.032,-0.020,-0.009,0.003,0.015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.000,1.037000e+07,-0.023,-0.022,-0.023,-0.025,-0.029,-0.029,-0.032,-0.035,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0.005,5.000000e-03,0.005,0.005,0.005,0.005,0.009,0.014,0.015,0.016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HR12587,164947007.0,NaN
0,-0.095,-9.500000e-02,-0.095,-0.096,-0.094,-0.095,-0.122,-0.154,-0.186,-0.200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HR16305,164947007.0,NaN
1,0.005,5.000000e-03,0.005,0.006,0.004,0.005,0.039,0.079,0.109,0.120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HR16305,164947007.0,NaN
0,-0.010,-1.000000e-02,-0.011,-0.008,-0.014,-0.010,0.065,0.089,0.073,0.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HR06450,164947007.0,NaN
